In [1]:
import os
import sys

root_path = os.path.abspath(os.path.join('..')) # <- adjust such that root_path always points at the root project dir (i.e. if current file is two folders deep, use '../..'). 
if root_path not in sys.path:
    sys.path.append(root_path)

import pandas as pd


import cleaning.data_cleaning as data_cleaning
# instantiate data cleaning object
cleaner = data_cleaning.DataCleaning()

import database_server.db_utilities as dbu

In [2]:
# read in raw data from csv files

path = "../data/scraped/fbref/match"
# read in all files in path
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
print(files)

# read in all individual files
df_list = []
for f in files:
    df_list.append(pd.read_csv(os.path.join(path, f), sep=';'))
# concatenate all files
df = pd.concat(df_list, ignore_index=True)
print(df.shape)


['league11_ssy2017_cols170_rows760.csv', 'league11_ssy2018_cols170_rows760.csv', 'league11_ssy2019_cols170_rows760.csv', 'league11_ssy2020_cols170_rows760.csv', 'league11_ssy2021_cols170_rows760.csv', 'league11_ssy2022_cols170_rows760.csv', 'league12_ssy2017_cols170_rows760.csv', 'league12_ssy2018_cols170_rows760.csv', 'league12_ssy2019_cols170_rows760.csv', 'league12_ssy2020_cols170_rows760.csv', 'league12_ssy2021_cols170_rows760.csv', 'league12_ssy2022_cols170_rows760.csv', 'league13_ssy2017_cols170_rows762.csv', 'league13_ssy2018_cols170_rows762.csv', 'league13_ssy2019_cols170_rows760.csv', 'league13_ssy2020_cols170_rows762.csv', 'league13_ssy2021_cols170_rows762.csv', 'league13_ssy2022_cols170_rows760.csv', 'league20_ssy2017_cols170_rows614.csv', 'league20_ssy2018_cols170_rows614.csv', 'league20_ssy2019_cols170_rows614.csv', 'league20_ssy2020_cols170_rows614.csv', 'league20_ssy2021_cols170_rows614.csv', 'league20_ssy2022_cols170_rows612.csv', 'league9_ssy2017_cols170_rows760.csv', 

In [3]:
# clean for insert into matchstats table
df = cleaner.clean_raw_matchstats_for_db(df)


Dropped 18 rows due to missing values in columns Index(['team_id', 'opponent_id', 'match_id'], dtype='object').
Dropped 598 rows due to missing values in more than 80.0% of columns.


In [7]:
df.shape

(21314, 153)

In [8]:
# get db id tables to create mapping dicts for ids
# leagues
leagues_df = dbu.select_query("SELECT * FROM leagues;")
leagues_dict = dict(zip(leagues_df['fbref_id'].astype(int), leagues_df['id']))
# teams
teams_df = dbu.select_query("SELECT * FROM teams;")
teams_dict = dict(zip(teams_df['fbref_id'], teams_df['id'].astype(int)))
# matches
matches_df = dbu.select_query("SELECT * FROM matches;")
matches_dict = dict(zip(matches_df['fbref_id'], matches_df['id'].astype(int)))

In [11]:
df[['match_id', 'league_id', 'opponent_id', 'team_id']].head()

,match_id,league_id,opponent_id,team_id
0,58,5,44,48
1,100,5,30,48
2,135,5,33,48
3,214,5,37,48
4,236,5,56,48


In [10]:
# replace fbref ids with db ids
df['league_id'] = df['league_id'].map(leagues_dict)
df['team_id'] = df['team_id'].map(teams_dict)
df['opponent_id'] = df['opponent_id'].map(teams_dict)
df['match_id'] = df['match_id'].map(matches_dict)


In [12]:
# inserts
# inserts (since we have many we reuse the cursor)
conn = dbu.get_conn(type='DB_su')
cursor = conn.cursor()
# iterate over df rows
counter = 0
query_str = ""
for i, r in df.iterrows():
    counter += 1
    colnames_str = ', '.join([c for c in df.columns])
    values_str = ', '.join([(('\'' + v + '\'' if type(v)==str else str(v)) if not pd.isna(v) else 'NULL') for v in r.values])
    query_str += f"""INSERT INTO matchstats ({colnames_str}) VALUES ({values_str});"""
    #print(query_str)
    # insert
    """
    cursor.execute(query_str)
    status_msg = cursor.statusmessage # get response
    conn.commit() # commit changes
    print(status_msg)
    """

    
    if counter % 100 == 0: # send in batches to avoid sending too many requests to the server
        cursor.execute(query_str)
        status_msg = cursor.statusmessage # get response
        conn.commit() # commit changes
        query_str = "" # reset query string
        print(status_msg)
        print(f"progress: {counter}/{df.shape[0]}")
        
# insert remaining rows
cursor.execute(query_str if query_str.endswith(';') else query_str + ';')
status_msg = cursor.statusmessage # get response
conn.commit() # commit changes
print(status_msg)
print(f"progress: {counter}/{df.shape[0]}")


cursor.close()
conn.close()



INSERT 0 1
progress: 100/21314
INSERT 0 1
progress: 200/21314
INSERT 0 1
progress: 300/21314
INSERT 0 1
progress: 400/21314
INSERT 0 1
progress: 500/21314
INSERT 0 1
progress: 600/21314
INSERT 0 1
progress: 700/21314
INSERT 0 1
progress: 800/21314
INSERT 0 1
progress: 900/21314
INSERT 0 1
progress: 1000/21314
INSERT 0 1
progress: 1100/21314
INSERT 0 1
progress: 1200/21314
INSERT 0 1
progress: 1300/21314
INSERT 0 1
progress: 1400/21314
INSERT 0 1
progress: 1500/21314
INSERT 0 1
progress: 1600/21314
INSERT 0 1
progress: 1700/21314
INSERT 0 1
progress: 1800/21314
INSERT 0 1
progress: 1900/21314
INSERT 0 1
progress: 2000/21314
INSERT 0 1
progress: 2100/21314
INSERT 0 1
progress: 2200/21314
INSERT 0 1
progress: 2300/21314
INSERT 0 1
progress: 2400/21314
INSERT 0 1
progress: 2500/21314
INSERT 0 1
progress: 2600/21314
INSERT 0 1
progress: 2700/21314
INSERT 0 1
progress: 2800/21314
INSERT 0 1
progress: 2900/21314
INSERT 0 1
progress: 3000/21314
INSERT 0 1
progress: 3100/21314
INSERT 0 1
progre

In [13]:
# check table
dbu.select_query("SELECT * FROM matchstats LIMIT 5;")

,venue,result,gf,ga,xg,xga,attendance,captain,formation,referee,...,misc_performance_crdr,misc_performance_2crdy,misc_performance_fls,misc_performance_fld,misc_performance_off,misc_performance_og,misc_performance_recov,misc_aerialduels_won,misc_aerialduels_lost,misc_aerialduels_won_perc
0,Home,W,3,0,1.1,1.4,39226,Gianluigi Buffon,4-2-3-1,Fabio Maresca,...,0,0,13,10,2,0,57,8,9,47.1
1,Away,W,4,2,2.1,1.2,26296,Gianluigi Buffon,4-2-3-1,Luca Banti,...,0,0,8,12,0,1,43,10,8,55.6
2,Home,W,3,0,1.3,0.6,39457,Stephan Lichtsteiner,4-3-3,Michael Fabbri,...,0,0,15,6,4,0,50,10,10,50.0
3,Away,W,3,1,1.0,1.4,21584,Gianluigi Buffon,4-2-3-1,Davide Massa,...,0,0,14,15,1,0,56,13,7,65.0
4,Home,W,1,0,0.9,0.4,35652,Andrea Barzagli,4-2-3-1,Daniele Doveri,...,0,0,10,23,3,0,54,9,8,52.9


In [9]:
# delete from table
dbu.manipulating_query("DELETE FROM matchstats;")

'DELETE 3900'

In [20]:
# check full table
matchstats_df = dbu.select_query("SELECT * FROM matchstats;")

In [19]:
for col, dt in zip(matchstats_df.columns, matchstats_df.dtypes):
    print(col, dt)

venue object
result object
gf int64
ga int64
xg float64
xga float64
attendance float64
captain object
formation object
referee object
season_str object
league_id int64
team_id int64
opponent_id int64
match_id int64
shooting_standard_gls int64
shooting_standard_sh int64
shooting_standard_sot int64
shooting_standard_sot_perc float64
shooting_standard_g_per_sh float64
shooting_standard_g_per_sot float64
shooting_standard_dist float64
shooting_standard_fk int64
shooting_standard_pk int64
shooting_standard_pkatt int64
shooting_expected_xg float64
shooting_expected_npxg float64
shooting_expected_npxg_per_sh float64
shooting_expected_g_minus_xg float64
shooting_expected_npg_minus_xg float64
keeper_performance_sota int64
keeper_performance_saves int64
keeper_performance_save_perc float64
keeper_performance_cs float64
keeper_performance_psxg float64
keeper_performance_psxg_plus_minus float64
keeper_penaltykicks_pkatt float64
keeper_penaltykicks_pka float64
keeper_penaltykicks_pksv float64
keepe

In [4]:
# drop table
conn = dbu.get_conn(type='DB_su')
resp = dbu.manipulating_query("DROP TABLE matchstats;", conn=conn)
conn.close()
print(resp)

DROP TABLE


In [5]:
# create table
query_str = """
CREATE TABLE matchstats (
    venue text,
    result matchresult,
    gf int,
    ga int,
    xg float,
    xga float,
    attendance int,
    captain text,
    formation text,
    referee text,
    season_str text,
    league_id int REFERENCES leagues (id) ,
    team_id int REFERENCES teams (id) NOT NULL,
    opponent_id int REFERENCES teams (id) NOT NULL,
    match_id int REFERENCES matches (id) NOT NULL,
    shooting_standard_gls int,
    shooting_standard_sh int,
    shooting_standard_sot int,
    shooting_standard_sot_perc float,
    shooting_standard_g_per_sh float,
    shooting_standard_g_per_sot float,
    shooting_standard_dist float,
    shooting_standard_fk int,
    shooting_standard_pk int,
    shooting_standard_pkatt int,
    shooting_expected_npxg float,
    shooting_expected_npxg_per_sh float,
    shooting_expected_g_minus_xg float,
    shooting_expected_npg_minus_xg float,
    keeper_performance_sota int,
    keeper_performance_saves int,
    keeper_performance_save_perc float,
    keeper_performance_cs int,
    keeper_performance_psxg float,
    keeper_performance_psxg_plus_minus float,
    keeper_penaltykicks_pkatt int,
    keeper_penaltykicks_pka int,
    keeper_penaltykicks_pksv int,
    keeper_penaltykicks_pkm int,
    keeper_launched_cmp int,
    keeper_launched_att int,
    keeper_launched_cmp_perc float,
    keeper_passes_att int,
    keeper_passes_thr int,
    keeper_passes_launch_perc float,
    keeper_passes_avglen float,
    keeper_goalkicks_att int,
    keeper_goalkicks_launch_perc float,
    keeper_goalkicks_avglen float,
    keeper_crosses_opp int,
    keeper_crosses_stp int,
    keeper_crosses_stp_perc float,
    keeper_sweeper_number_opa int,
    keeper_sweeper_avgdist float,
    passing_total_cmp int,
    passing_total_att int,
    passing_total_cmp_perc float,
    passing_total_totdist float,
    passing_total_prgdist float,
    passing_short_cmp int,
    passing_short_att int,
    passing_short_cmp_perc float,
    passing_medium_cmp int,
    passing_medium_att int,
    passing_medium_cmp_perc float,
    passing_long_cmp int,
    passing_long_att int,
    passing_long_cmp_perc float,
    passing_attacking_ast int,
    passing_attacking_xag float,
    passing_attacking_xa float,
    passing_attacking_kp int,
    passing_attacking_1_per_3 int,
    passing_attacking_ppa int,
    passing_attacking_crspa int,
    passing_attacking_prgp int,
    passing_types_passtypes_live int,
    passing_types_passtypes_dead int,
    passing_types_passtypes_fk int,
    passing_types_passtypes_tb int,
    passing_types_passtypes_sw int,
    passing_types_passtypes_crs int,
    passing_types_passtypes_ti int,
    passing_types_passtypes_ck int,
    passing_types_cornerkicks_in int,
    passing_types_cornerkicks_out int,
    passing_types_cornerkicks_str int,
    passing_types_outcomes_off int,
    passing_types_outcomes_blocks int,
    gca_scatypes_sca int,
    gca_scatypes_passlive int,
    gca_scatypes_passdead int,
    gca_scatypes_to int,
    gca_scatypes_sh int,
    gca_scatypes_fld int,
    gca_scatypes_def int,
    gca_gcatypes_gca int,
    gca_gcatypes_passlive int,
    gca_gcatypes_passdead int,
    gca_gcatypes_to int,
    gca_gcatypes_sh int,
    gca_gcatypes_fld int,
    gca_gcatypes_def int,
    defense_tackles_tkl int,
    defense_tackles_tklw int,
    defense_tackles_def3rd int,
    defense_tackles_mid3rd int,
    defense_tackles_att3rd int,
    defense_challenges_tkl int,
    defense_challenges_att int,
    defense_challenges_tkl_perc float,
    defense_challenges_lost int,
    defense_blocks_blocks int,
    defense_blocks_sh int,
    defense_blocks_pass int,
    defense_general_int int,
    defense_general_tkl_plus_int int,
    defense_general_clr int,
    defense_general_err int,
    possession_general_poss float,
    possession_touches_touches int,
    possession_touches_defpen int,
    possession_touches_def3rd int,
    possession_touches_mid3rd int,
    possession_touches_att3rd int,
    possession_touches_attpen int,
    possession_touches_live int,
    possession_takeons_att int,
    possession_takeons_succ int,
    possession_takeons_succ_perc float,
    possession_takeons_tkld int,
    possession_takeons_tkld_perc float,
    possession_carries_carries int,
    possession_carries_totdist float,
    possession_carries_prgdist float,
    possession_carries_prgc int,
    possession_carries_1_per_3 int,
    possession_carries_cpa int,
    possession_carries_mis int,
    possession_carries_dis int,
    possession_receiving_rec int,
    possession_receiving_prgr int,
    misc_performance_crdy int,
    misc_performance_crdr int,
    misc_performance_2crdy int,
    misc_performance_fls int,
    misc_performance_fld int,
    misc_performance_off int,
    misc_performance_og int,
    misc_performance_recov int,
    misc_aerialduels_won int,
    misc_aerialduels_lost int,
    misc_aerialduels_won_perc float,
    PRIMARY KEY (match_id, team_id) -- composite primary key
);
"""

conn = dbu.get_conn(type='DB_su')
resp = dbu.manipulating_query(query_str, conn=conn)
conn.close()
print(resp)

CREATE TABLE


In [6]:
# grant access
query_str = """
GRANT ALL ON ALL TABLES IN SCHEMA public TO project_client; 
GRANT ALL ON ALL SEQUENCES IN SCHEMA public TO project_client;
"""
conn = dbu.get_conn(type='DB_su')
resp = dbu.manipulating_query(query_str, conn=conn)
conn.close()
print(resp)

GRANT
